<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/data_postprocess/prediction_postprocess_triviaqa_sliced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import numpy as np
import pandas as pd

#from google.colab import data_table
#data_table.enable_dataframe_formatter()

In [15]:
# Some important file locations and constants

project_root = "/content/drive/MyDrive/w266 NLP Final Project/"
prediction_folder = project_root + "Predictions_3/checkpoint/"
data_folder = project_root + "Data/"
processed_folder = project_root + "Predictions_3/"

datasets = ['valid_pairs_v1.csv', 'valid_pairs_v2.csv', 'valid_pairs_d1.csv']

validation_set_size = {}

In [4]:
for dataset in datasets:
  df = pd.read_csv(f"{data_folder}triviaqa/{dataset}")
  validation_set_size [dataset.split(".")[0]] = df.shape[0]

In [5]:
validation_set_size

{'valid_pairs_v1': 493, 'valid_pairs_v2': 492, 'valid_pairs_d1': 985}

In [6]:
file_list = glob.glob(prediction_folder + "*.csv")

In [7]:
file_list

['/content/drive/MyDrive/w266 NLP Final Project/Predictions_3/checkpoint/predictions.T5_base_pt_long.amalgam.triviaqa_v1.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions_3/checkpoint/predictions.T5_base_pt_long.amalgam.triviaqa_v2.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions_3/checkpoint/predictions.T5_base_pt_long.amalgam.triviaqa_d1.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions_3/checkpoint/predictions.T5_base_pt_long.nq.triviaqa_v1.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions_3/checkpoint/predictions.T5_base_pt_long.nq.triviaqa_v2.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions_3/checkpoint/predictions.T5_base_pt_long.nq.triviaqa_d1.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions_3/checkpoint/predictions.T5_base_pt_long.squad.triviaqa_v1.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions_3/checkpoint/predictions.T5_base_pt_long.squad.triviaqa_v2.csv',
 '/

In [8]:
len(file_list)

30

In [9]:
df = pd.DataFrame([(os.path.basename(x), len (pd.read_csv(x))) for x in file_list], columns=['name', 'count']).sort_values(by=['count'])
df

,name,count
10,predictions.T5_base_pt_long.triviaqa.triviaqa_...,492
1,predictions.T5_base_pt_long.amalgam.triviaqa_v...,492
19,predictions.bart_base_pt_long.nq.triviaqa_v2.csv,492
4,predictions.T5_base_pt_long.nq.triviaqa_v2.csv,492
16,predictions.bart_base_pt_long.amalgam.triviaqa...,492
7,predictions.T5_base_pt_long.squad.triviaqa_v2.csv,492
25,predictions.bart_base_pt_long.triviaqa.triviaq...,492
28,predictions.bart_base_pt_long.quac.triviaqa_v2...,492
22,predictions.bart_base_pt_long.squad.triviaqa_v...,492
13,predictions.T5_base_pt_long.quac.triviaqa_v2.csv,492


In [10]:
for name in file_list:
  df = pd.read_csv(name)
  df = df.replace(r'^\s*$', np.nan, regex=True)
  count = df['prediction'].isna().sum()
  if count > 0:
    print (f"{os.path.basename(name)} empty predictions: {count}")

In [11]:
for file in file_list:
  model, training, validation = os.path.basename(file).split(".")[1:4]
  df = pd.read_csv(file)
  print (f"{model} {training} validation {validation}", end=" ")
  if df.shape[0]==validation_set_size[validation]:
    print ("proper length")
  else:
    print ("BAD LENGTH")  

T5_base_pt_long amalgam validation triviaqa_v1 

KeyError: ignored

In [16]:
for file in file_list:
  df = pd.read_csv(file, usecols=['context', 'answer', 'target', 'prediction'])
  df = df.replace(r'^\s*$', np.nan, regex=True)
  df['prediction'] = df['prediction'].fillna('NO PREDICTION MADE')
  output_file = f"{processed_folder}{os.path.basename(file)}"
  df.to_csv(output_file, index=False)

In [17]:
for file in file_list:
  model, training, validation = os.path.basename(file).split(".")[1:4]
  test_file = f"{processed_folder}{os.path.basename(file)}"
  df = pd.read_csv(test_file)
  print(df.keys())

Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'targe

In [18]:
for file in file_list:
  model, training, validation = os.path.basename(file).split(".")[1:4]
  test_file = f"{processed_folder}{os.path.basename(file)}"
  df = pd.read_csv(test_file)
  df = df[df['prediction'] == "NO PREDICTION MADE"]
  if df.shape[0]: 
    print (test_file, df.shape[0])
    print (df.index)